In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [55]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [22]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [23]:
train_df[train_df["target"] == 0]["text"].values[1]
train_df[train_df["target"] == 1]["text"].values[1]

'I love fruits'

'Forest fire near La Ronge Sask. Canada'

In [24]:
count_vectorizer = feature_extraction.text.CountVectorizer()
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [25]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [27]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

In [28]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [29]:
## here are lots of ways to potentially improve on this 
## (TFIDF, LSA, LSTM / RNNs, the list is long!)
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59421842, 0.56498283, 0.64113893])

In [30]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [32]:
sample_submission = pd.read_csv("data/sample_submission.csv")

In [33]:
sample_submission["target"] = clf.predict(test_vectors)

In [34]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [35]:
sample_submission.to_csv("submission.csv", index=False)

In [83]:
###############################
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

train_corpus = count_vectorizer.fit_transform(train_df["text"])
test_corpus = count_vectorizer.transform(test_df["text"])

onehot = Binarizer()
# tfidf = TfidfVectorizer()

train_corpus = onehot.fit_transform(train_corpus)
test_corpus = onehot.fit_transform(test_corpus)

clf = MultinomialNB()
scores = model_selection.cross_val_score(clf, train_corpus, train_df["target"], cv=3, scoring="f1")

In [84]:
scores

array([0.67208413, 0.65487528, 0.72844273])

In [82]:
scores

array([0.60455267, 0.59820161, 0.64034665])

In [85]:
clf.fit(train_corpus, train_df["target"])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [86]:
sample_submission["target"] = clf.predict(test_vectors)

In [87]:
sample_submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [88]:
sample_submission.to_csv("submission.csv", index=False)